### Test the usage of random forest
Aug 7, 2020

In [ ]:
import numpy as np
import glob
import os
import pandas as pd
import pickle

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# data_dir='/global/cfs/cdirs/dasrepo/vpa/supernova_cnn/data/autoscan_data/data/'


In [ ]:
fname='/global/cfs/cdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/autoscan_features.3.csv'
# df=pd.read_csv(fname,error_bad_lines=False)

df_full=pd.read_csv(fname,sep=',',comment='#')


### Convert BAND to integers
bm = {'g':0, 'r':1, 'i':2, 'z':3}
df_full['BAND']=np.array([bm[i] for i in df_full.BAND.values])
df_full.head()

In [ ]:
df=df_full[:]
df.shape

y_cols=['OBJECT_TYPE']
# Drop columns with Nans
# df.columns[df_full.isna().any().values]
reject_cols=['A_REF', 'B_REF', 'MAG_REF', 'MAG_REF_ERR', 'NN_DIST_RENORM']
x_cols=[i for i in df.columns if i not in ['OBJECT_TYPE','ID']+reject_cols]


In [ ]:
# X=df[x_cols]
# y=df[y_cols]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
### Get IDs for test data
results_dir='/global/project/projectdirs/dasrepo/vpa/supernova_cnn/data/results_data/results/'
fname='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Supernova-cnn/cnn_supernova/4_train_random_forest/train_ids.train'
train_ids=np.loadtxt(fname,dtype=np.int32)
fname=results_dir+'id_test_3.test'
test_ids=np.loadtxt(fname,dtype=np.int32)


In [ ]:
ID_list.shape

In [ ]:
# Get train data
df_temp=df[df.ID.isin(train_ids)]
X_train,y_train=df_temp[x_cols],df_temp[y_cols]

# Get test data
df_temp=df[df.ID.isin(test_ids)]
X_test,y_test=df_temp[x_cols],df_temp[y_cols]



In [ ]:
# X_train.shape

In [ ]:
# df.dropna().shape

In [ ]:
# ### Load importance of features 

# fle='features.txt'
# a1=np.loadtxt(fle,delimiter=',',dtype=str)

# weight_dict1={}
# for i in range(a1.shape[0]):
#     key,val=a1[i][0],a1[i][1]
#     weight_dict1[key.upper()]=val

# # [i for i in df_full.columns if i not in weight_dict.keys()],[i for i in weight_dict.keys() if i not in df.columns]
    
# key_maps={'R_APER_PSF':'GFLUX','DIFFSUM':'DIFFSUMRN','MIN_DIST_TO_EDGE':'MIN_DISTANCE_TO_EDGE_IN_NEW','NUMNEG':'NUMNEGRN','BANDNUM':'BAND'}
# # Renaming keys to match those in DataFrame
# weight_dict={}
# for key in weight_dict1.keys():
#     if key not in key_maps:
#         weight_dict[key]=float(weight_dict1[key])
#     else : 
#         weight_dict[key_maps[key]]=float(weight_dict1[key])

# # weight_dict

In [ ]:
model=RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)


In [ ]:
y_predict=model.predict_proba(X_test)

In [ ]:
model.score(X_test,y_test.values)

In [ ]:
y_test.values.shape

In [ ]:
for i in range(100,110):
    print(y_test.values[i,0],y_predict[i,0],y_predict[i,1])


In [ ]:
df_save=pd.DataFrame([])

In [ ]:
df_save['ID']=df_temp.ID.values
df_save['m_rf']=y_predict[:,1]

In [ ]:
df_save.to_csv('rf_preds.csv',index=False)

In [ ]:
! ls 